# Implementing our RAG Application

We've walked through the process of ingesting our data. Now we want to setup our search application.

There are multiple ways of implementing search that we won't be addressing in this workshop.

We're going to implement a similarity search. We can choose to use the OpenSearch®️ SDK or we can use LangChain.

## Similarity Search with Langchain

LangChain allows us to connect our existing configurations across our platform to ensure that we settings are consistent. This is paramount to the success of your similarity search in that if you're search parameters differ from the values you selected to load data from, you can run into some issues.

LangChain gives us the ability to access a [preexisting OpenSearch instance](https://python.langchain.com/docs/integrations/vectorstores/opensearch/#using-a-preexisting-opensearch-instance).

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch

load_dotenv()

embeddings = HuggingFaceEmbeddings()
vector_search = OpenSearchVectorSearch(
    index_name=os.getenv("INDEX_NAME"),
    embedding_function=embeddings,
    opensearch_url=os.getenv("OPENSEARCH_SERVICE_URI"),
)

query = "How can I be productive"

results = vector_search.similarity_search(
    query,
    vector_field="content_vector",
    text_field="content",
    metadata_field="*",
)

pprint(results[0].page_content)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="llama3")
prompt = ChatPromptTemplate.from_template("""
        Offer supportive advice for the question {query} with supporting quotes from 
        "{docs}".
                                          
        Don't include quotes from other sources.
""")

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"query": query, "docs": "\n".join([result.page_content for result in results])}))
print("-------------------")
print(f'Here are some episodes that might help you with "{query}":')
episodes = set()
for result in results:
    episodes.add(f"{result.metadata["title"]} - {result.metadata["url"]}")
print("\n".join(episodes))

This is good but we probably don't want to use Ollama in our production environment. Let look at how easy it is to use a different model.

> NOTE: ⚠️ The next block uses the OpenAI API which cannot be used without an API key which you will need to pay for.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """Offer supportive advice for the question {query} with supporting quotes from 
     ---
     {docs}
     ---
     Wrap quotes in quotation marks. Don't include quotes from other sources.
     If there are no documents to quote, say \"I don't have any information on that.\"""")
    ,
    ("user",
     "{query}"),
])

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"query": query, "docs": "\n".join([result.page_content for result in results])}))
print("-------------------")
print(f'Here are some episodes that might help you with "{query}":')
episodes = set()
for result in results:
    episodes.add(f"{result.metadata["title"]} - {result.metadata["url"]}")
print("\n".join(episodes))